# CAPSTONE PROJECT (CONTECH UNIVERSITY)
## COURSE INSTRUCTOR: DR SUMAN SAHA
## COLLABORATORS: 
#####   ZAKRIA SAAD
#####   TINASHE HAFE
#####   TRYMORE NCUBE

# PROJECT TOPIC: FRAUD DETECTION IN ELECTRICITY AND GAS CONSUMPTION


# This Notebook is developed for feature Engineering on fraud detection in electricity and gas consumption dataset

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the dataset

In [3]:
client_train=pd.read_csv('/content/drive/MyDrive/client_train.csv')
invoice_train=pd.read_csv('/content/drive/MyDrive/invoice_train.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Invoice column manipulation

In [4]:
invoice_train['invoice_date']=pd.to_datetime(invoice_train['invoice_date'])
invoice_train["year"] = invoice_train["invoice_date"].dt.year
invoice_train["month"] = invoice_train["invoice_date"].dt.month

# Taking Cummulative summation of consomation levels and indexes

# Cummulative sum can helpful in indicating sudden decrement or increments 

In [5]:
invoice_train_cumsum = invoice_train.copy()

cols_to_cumsum = [
    'consommation_level_1',
    'consommation_level_2',
    'consommation_level_3',
    'consommation_level_4',
    'old_index',
    'new_index'
]

# Calculate cumulative sum for each group separately
groups = invoice_train.groupby('client_id')[cols_to_cumsum].cumsum()

# Rename the columns to indicate that they are cumulative sums
groups.columns = [f'cumsum_{col}' for col in cols_to_cumsum]

# Join the cumulative sums with the original DataFrame
invoice_train_cumsum = invoice_train_cumsum.join(groups)

# Sort the DataFrame by client_id and reset the index
invoice_train_cumsum = invoice_train_cumsum.sort_values('client_id').reset_index(drop=True)


## This is the most important work of this project
# Features Used for generating new features
###Electric and Gas Consommation levels (1-4)
###Old Index
###New Index
###Tariff Type
###Counter Status

# Methods Used for generating new features
###Cumulative Sum
###Measures of Central Tendencies (Mean, mode, median)
###Measures of Spread (range, standard deviation, variance)
###Group By methods in Pandas 



In [6]:
from collections import Counter

def find_mode(x):
    counter = Counter(x)
    mode = counter.most_common(1)[0][0]
    return mode

summary_invoice_train = (
    invoice_train_cumsum.loc[:, ~invoice_train_cumsum.columns.isin(["counter_code", "counter_number"])].groupby(["client_id", "counter_type"]).agg(avg_consom_l_1=("consommation_level_1", "mean"),
         var_consom_l_1=("consommation_level_1", "var"),
         sd_consom_l_1=("consommation_level_1", "std"),
         median_consom_l_1=("consommation_level_1", "median"),
         mode_consom_l_1=("consommation_level_1", find_mode),
         avg_diff_consom_l_1=("consommation_level_1", lambda x: np.mean(np.diff(x))),
         range_consom_l_1=("consommation_level_1", lambda x: np.max(x) - np.min(x)),
         sd_cumsum_consommation_level_1=("cumsum_consommation_level_1", "std"),
         avg_cumsum_consommation_level_1=("cumsum_consommation_level_1", "mean"),
         median_cumsum_consommation_level_1=("cumsum_consommation_level_1", "median"),
         avg_consom_l_2=("consommation_level_2", "mean"),
         var_consom_l_2=("consommation_level_2", "var"),
         sd_consom_l_2=("consommation_level_2", "std"),
         median_consom_l_2=("consommation_level_2", "median"),
         mode_consom_l_2=("consommation_level_2", find_mode),
         avg_diff_consom_l_2=("consommation_level_2", lambda x: np.mean(np.diff(x))),
         range_consom_l_2=("consommation_level_2", lambda x: np.max(x) - np.min(x)),
         sd_cumsum_consommation_level_2=("cumsum_consommation_level_2", "std"),
         avg_cumsum_consommation_level_2=("cumsum_consommation_level_2", "mean"),
         median_cumsum_consommation_level_2=("cumsum_consommation_level_2", "median"),
         avg_consom_l_3=("consommation_level_3", "mean"),
         var_consom_l_3=("consommation_level_3", "var"),
         sd_consom_l_3=("consommation_level_3", "std"),
         median_consom_l_3=("consommation_level_3", "median"),
         mode_consom_l_3=("consommation_level_3", find_mode),
         avg_diff_consom_l_3=("consommation_level_3", lambda x: np.mean(np.diff(x))),
         range_consom_l_3=("consommation_level_3", lambda x: np.max(x) - np.min(x)),
         sd_cumsum_consommation_level_3=("cumsum_consommation_level_3", "std"),
         avg_cumsum_consommation_level_3=("cumsum_consommation_level_3", "mean"),
         median_cumsum_consommation_level_3=("cumsum_consommation_level_3", "median"),
         avg_consom_l_4=("consommation_level_4", "mean"),
         var_consom_l_4=("consommation_level_4", "var"),
         sd_consom_l_4 =("consommation_level_4", "std"),
         median_consom_l_4 = ('consommation_level_4',"median"),
         mode_consom_l_4 = ("consommation_level_4", find_mode),
         avg_diff_consom_l_4 = ("consommation_level_4", lambda x: np.mean(np.diff(x))),
         range_consom_l_4 = ("consommation_level_1", lambda x: np.max(x) - np.min(x)),
         sd_cumsum_consommation_level_4=("cumsum_consommation_level_4", "std"),
         avg_cumsum_consommation_level_4=("cumsum_consommation_level_4", "mean"),
         median_cumsum_consommation_level_4=("cumsum_consommation_level_4", "median"),
         avg_diff_old_index=('old_index', lambda x: x.diff().mean()),
         var_old_index=('old_index',lambda x: x.var()),
         avg_diff_new_index=('new_index', lambda x: x.diff().mean()),
         var_new_index=('new_index', lambda x: x.var()),
         diff_avg_new_old_index=('new_index', lambda x: x.diff().mean() - x.diff().mean()),
         range_old_index=('old_index', lambda x: np.max(x) - np.min(x)),
         range_new_index=('new_index', lambda x: np.max(x) - np.min(x)),
         min_old_index=('old_index', lambda x: np.min(x)),
         min_new_index=('new_index', lambda x: np.min(x)),
         max_old_index=('old_index', 'max'),
         max_new_index=('new_index', 'max'),
         sd_old_index=('old_index', 'std'),
         sd_new_index=('new_index', 'std'),
         sd_cumsum_old_index=('cumsum_old_index', 'std'),
         avg_cumsum_old_index=('cumsum_old_index', 'mean'),
         median_cumsum_old_index=('cumsum_old_index', 'median'),
         sd_cumsum_new_index=('cumsum_new_index', 'std'),
         avg_cumsum_new_index=('cumsum_new_index', 'mean'),
         median_cumsum_new_index=('cumsum_new_index', 'median'),
         count_counter_coefficient=('counter_coefficient', 'size'),
         mean_counter_coefficient=('counter_coefficient', 'mean'),
         count_invoice_date=('invoice_date', 'size'),
         mode_reading_remarque=('reading_remarque', find_mode),
         mode_months_number=('months_number', find_mode),
         mode_counter_statue=('counter_statue', find_mode))
)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Data Preprocessing After generating new features

In [7]:
summary_invoice_train=summary_invoice_train.reset_index()

In [8]:
print(summary_invoice_train.shape)
print(summary_invoice_train['client_id'].duplicated().sum())

(196869, 67)
61376


# Pivoting data on counter type

In [9]:
summary_invoice_train_pivot = summary_invoice_train.pivot(index='client_id', columns='counter_type')

In [11]:
summary_invoice_train_pivotted = summary_invoice_train_pivot.loc[:, ~(summary_invoice_train_pivot.columns.get_level_values(0).isin(['ELEC', 'GAZ']))]
summary_invoice_train_final = summary_invoice_train_pivotted.apply(pd.to_numeric, errors='coerce').fillna(0)

In [12]:
from pandas.api.types import CategoricalDtype
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# left join client_train and summary_invoice_train_wider on "client_id"
# merge client_train and summary_invoice_train_final on client_id
train_full = pd.merge(client_train, summary_invoice_train_final, on="client_id", how="left")

# convert creation_date to datetime and extract month
train_full["creation_date"] = pd.to_datetime(train_full["creation_date"], format="%d/%m/%Y")
train_full["month"] = train_full["creation_date"].dt.month

# reverse the categories of target, convert it to a categorical variable and select all numeric columns except target and creation_date
train_full["target"] = train_full["target"][::-1]
train_full["target"] = train_full["target"].astype('category')
train_full["target"] = train_full["target"].cat.set_categories([0, 1])
train_full["target"] = train_full["target"].cat.reorder_categories([1, 0], ordered=True)

#train_full["target"] = train_full["target"].astype(CategoricalDtype(ordered=True)).cat.reorder_categories(["0", "1"], ordered=True)
numeric_cols = train_full.select_dtypes(include='number').columns.tolist()
if "target" in numeric_cols:
    numeric_cols.remove("target")
if "creation_date" in numeric_cols:
    numeric_cols.remove("creation_date")
train_full = train_full[numeric_cols]
train_full["target"] = client_train["target"]


<ipython-input-12-890d73a67151>:8: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  train_full = pd.merge(client_train, summary_invoice_train_final, on="client_id", how="left")
/usr/local/lib/python3.8/dist-packages/pandas/core/common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
<ipython-input-12-890d73a67151>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_full["target"] = clien

In [13]:
train_full["target"].value_counts()

0.0    127927
1.0      7566
Name: target, dtype: int64

In [14]:
train_full["target"].unique()

array([0., 1.])

In [16]:
train_full

,disrict,client_catg,region,"(avg_consom_l_1, ELEC)","(avg_consom_l_1, GAZ)","(var_consom_l_1, ELEC)","(var_consom_l_1, GAZ)","(sd_consom_l_1, ELEC)","(sd_consom_l_1, GAZ)","(median_consom_l_1, ELEC)",...,"(count_invoice_date, GAZ)","(mode_reading_remarque, ELEC)","(mode_reading_remarque, GAZ)","(mode_months_number, ELEC)","(mode_months_number, GAZ)","(mode_counter_statue, ELEC)","(mode_counter_statue, GAZ)",month,target,client_id
0,60,11,101,352.400000,0.000000,96313.070588,0.000000,310.343472,0.000000,267.0,...,0.0,6.0,0.0,4.0,0.0,0.0,0.0,12,0.0,train_Client_0
1,69,11,107,557.540541,0.000000,39178.644144,0.000000,197.935960,0.000000,520.0,...,0.0,6.0,0.0,4.0,0.0,0.0,0.0,5,0.0,train_Client_1
2,62,11,301,798.611111,0.000000,264032.957516,0.000000,513.841374,0.000000,655.5,...,0.0,6.0,0.0,4.0,0.0,0.0,0.0,3,0.0,train_Client_10
3,69,11,105,1.200000,0.000000,13.010526,0.000000,3.607011,0.000000,0.0,...,0.0,6.0,0.0,4.0,0.0,0.0,0.0,7,0.0,train_Client_100
4,62,11,303,663.714286,0.000000,50549.142857,0.000000,224.831365,0.000000,770.0,...,0.0,9.0,0.0,4.0,0.0,0.0,0.0,10,0.0,train_Client_1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,62,11,304,0.000000,4.088235,0.000000,568.264706,0.000000,23.838303,0.0,...,34.0,6.0,6.0,4.0,4.0,0.0,0.0,7,0.0,train_Client_99995
135489,63,11,311,309.700000,67.904762,49830.852632,3465.190476,223.228252,58.865869,278.0,...,21.0,9.0,9.0,4.0,4.0,0.0,0.0,10,0.0,train_Client_99996
135490,63,11,311,405.000000,65.785714,26984.857143,686.181319,164.270683,26.195063,366.0,...,14.0,9.0,9.0,4.0,4.0,0.0,0.0,11,0.0,train_Client_99997
135491,60,11,101,300.000000,0.000000,20000.000000,0.000000,141.421356,0.000000,300.0,...,0.0,9.0,0.0,8.0,0.0,0.0,0.0,12,0.0,train_Client_99998


In [15]:
single_col=client_train['client_id']
train_full = train_full.join(single_col)

In [17]:
cols = list(train_full.columns)
cols = [cols[-1]] + cols[:-1]
train_full= train_full[cols]

/usr/local/lib/python3.8/dist-packages/pandas/core/common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/output.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   train_full.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
train_full

,client_id,disrict,client_catg,region,"(avg_consom_l_1, ELEC)","(avg_consom_l_1, GAZ)","(var_consom_l_1, ELEC)","(var_consom_l_1, GAZ)","(sd_consom_l_1, ELEC)","(sd_consom_l_1, GAZ)",...,"(count_invoice_date, ELEC)","(count_invoice_date, GAZ)","(mode_reading_remarque, ELEC)","(mode_reading_remarque, GAZ)","(mode_months_number, ELEC)","(mode_months_number, GAZ)","(mode_counter_statue, ELEC)","(mode_counter_statue, GAZ)",month,target
0,train_Client_0,60,11,101,352.400000,0.000000,96313.070588,0.000000,310.343472,0.000000,...,35.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,12,0.0
1,train_Client_1,69,11,107,557.540541,0.000000,39178.644144,0.000000,197.935960,0.000000,...,37.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,5,0.0
2,train_Client_10,62,11,301,798.611111,0.000000,264032.957516,0.000000,513.841374,0.000000,...,18.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,3,0.0
3,train_Client_100,69,11,105,1.200000,0.000000,13.010526,0.000000,3.607011,0.000000,...,20.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,7,0.0
4,train_Client_1000,62,11,303,663.714286,0.000000,50549.142857,0.000000,224.831365,0.000000,...,14.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,train_Client_99995,62,11,304,0.000000,4.088235,0.000000,568.264706,0.000000,23.838303,...,37.0,34.0,6.0,6.0,4.0,4.0,0.0,0.0,7,0.0
135489,train_Client_99996,63,11,311,309.700000,67.904762,49830.852632,3465.190476,223.228252,58.865869,...,20.0,21.0,9.0,9.0,4.0,4.0,0.0,0.0,10,0.0
135490,train_Client_99997,63,11,311,405.000000,65.785714,26984.857143,686.181319,164.270683,26.195063,...,22.0,14.0,9.0,9.0,4.0,4.0,0.0,0.0,11,0.0
135491,train_Client_99998,60,11,101,300.000000,0.000000,20000.000000,0.000000,141.421356,0.000000,...,2.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,12,0.0


# Fixing column names

In [19]:
train_full.columns = ['_'.join(col).strip() for col in train_full.columns.values]

# replace commas with underscores
train_full.columns = train_full.columns.str.replace(',', '_')
train_full.columns = train_full.columns.str.replace(' ', '')

In [20]:
train_full.columns

Index(['c_l_i_e_n_t___i_d', 'd_i_s_r_i_c_t', 'c_l_i_e_n_t___c_a_t_g',
       'r_e_g_i_o_n', 'avg_consom_l_1_ELEC', 'avg_consom_l_1_GAZ',
       'var_consom_l_1_ELEC', 'var_consom_l_1_GAZ', 'sd_consom_l_1_ELEC',
       'sd_consom_l_1_GAZ',
       ...
       'count_invoice_date_ELEC', 'count_invoice_date_GAZ',
       'mode_reading_remarque_ELEC', 'mode_reading_remarque_GAZ',
       'mode_months_number_ELEC', 'mode_months_number_GAZ',
       'mode_counter_statue_ELEC', 'mode_counter_statue_GAZ', 'm_o_n_t_h',
       't_a_r_g_e_t'],
      dtype='object', length=136)

In [33]:
train_full = train_full.rename(columns={
    'd_i_s_r_i_c_t': 'district',
    'c_l_i_e_n_t___c_a_t_g': 'client_category',
    'r_e_g_i_o_n': 'region',
    'm_o_n_t_h':'month',
    't_a_r_g_e_t':'target'
})

In [34]:
train_full

,c_l_i_e_n_t___i_d,district,client_category,region,avg_consom_l_1_ELEC,avg_consom_l_1_GAZ,var_consom_l_1_ELEC,var_consom_l_1_GAZ,sd_consom_l_1_ELEC,sd_consom_l_1_GAZ,...,count_invoice_date_ELEC,count_invoice_date_GAZ,mode_reading_remarque_ELEC,mode_reading_remarque_GAZ,mode_months_number_ELEC,mode_months_number_GAZ,mode_counter_statue_ELEC,mode_counter_statue_GAZ,month,target
0,train_Client_0,60,11,101,352.400000,0.000000,96313.070588,0.000000,310.343472,0.000000,...,35.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,12,0.0
1,train_Client_1,69,11,107,557.540541,0.000000,39178.644144,0.000000,197.935960,0.000000,...,37.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,5,0.0
2,train_Client_10,62,11,301,798.611111,0.000000,264032.957516,0.000000,513.841374,0.000000,...,18.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,3,0.0
3,train_Client_100,69,11,105,1.200000,0.000000,13.010526,0.000000,3.607011,0.000000,...,20.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,7,0.0
4,train_Client_1000,62,11,303,663.714286,0.000000,50549.142857,0.000000,224.831365,0.000000,...,14.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,train_Client_99995,62,11,304,0.000000,4.088235,0.000000,568.264706,0.000000,23.838303,...,37.0,34.0,6.0,6.0,4.0,4.0,0.0,0.0,7,0.0
135489,train_Client_99996,63,11,311,309.700000,67.904762,49830.852632,3465.190476,223.228252,58.865869,...,20.0,21.0,9.0,9.0,4.0,4.0,0.0,0.0,10,0.0
135490,train_Client_99997,63,11,311,405.000000,65.785714,26984.857143,686.181319,164.270683,26.195063,...,22.0,14.0,9.0,9.0,4.0,4.0,0.0,0.0,11,0.0
135491,train_Client_99998,60,11,101,300.000000,0.000000,20000.000000,0.000000,141.421356,0.000000,...,2.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,12,0.0


# final shape

In [35]:
train_full.shape

(135493, 136)